<a href="https://colab.research.google.com/github/marlenebriseno-sys/AAI2025/blob/main/Prompt_Chaining_for_a_Customer_Support_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# JCPenney Customer Support Prompt Chain

# --- Step 0: Mini FAQ we will use (JCPenney themed demo) ---
FAQ = {
    "credit_card": "For JCPenney credit card (regular or Mastercard) payments or billing issues, log in to your JCPenney Credit Card Account online. You can update your payment method under Billing → Update Payment Method.",
    "returns": "JCPenney accepts returns within 60 days of purchase. Bring your receipt or order confirmation, or start a return online under Orders → Select Item → Return.",
    "order_tracking": "To track your JCPenney order, log in to your account and go to Orders → Track Order. You’ll see the latest shipping status there."
}

# --- Step 1: Figure out what the user wants (intent) ---
def classify_intent(user_message):
    msg = user_message.lower()

    # credit card related
    if any(word in msg for word in ["credit card", "mastercard", "billing", "payment", "statement"]):
        return {"intent": "credit_card", "confidence": 0.92}

    # returns related
    if any(word in msg for word in ["return", "refund", "exchange"]):
        return {"intent": "returns", "confidence": 0.90}

    # order tracking
    if any(word in msg for word in ["track", "tracking", "order status", "shipping", "delivery"]):
        return {"intent": "order_tracking", "confidence": 0.88}

    return {"intent": "unknown", "confidence": 0.50}

# --- Step 2: Look up a helpful answer ---
def get_faq_snippet(intent):
    return FAQ.get(intent, "No matching FAQ found. Please connect to a JCPenney customer support agent for help.")

# --- Step 3: Ask a follow-up if needed ---
def make_clarifying_question(user_message):
    if "?" in user_message:
        return "no_clarification_needed"
    return "Could you tell me if this is about your JCPenney Credit Card, a return, or an order tracking request?"

# --- Step 4: Put together the final answer ---
def compose_final_answer(snippet, clarifying_question):
    answer = (
        "Thanks for contacting JCPenney Customer Support! Here's what to do:\n"
        f"- {snippet}\n"
        "If you run into trouble, reply here and we’ll be happy to assist further."
    )
    if clarifying_question != "no_clarification_needed":
        answer += f"\n\nBefore we proceed: {clarifying_question}"
    return answer

# --- Step 5: Quick quality check ---
def quality_check(answer_text):
    is_complete = "complete" if "reply here" in answer_text else "incomplete"
    suggestion = "Add a direct link to the JCPenney Help Center for more details."
    return {"rating": is_complete, "suggestion": suggestion}

# --- Step 6: Run the full support flow ---
def run_support_chain(user_message):
    print("USER MESSAGE:", user_message)
    print("-" * 60)

    intent_info = classify_intent(user_message)
    print("Step 1) Intent:", intent_info)

    snippet = get_faq_snippet(intent_info["intent"])
    print("Step 2) FAQ Snippet:", snippet)

    clarifying = make_clarifying_question(user_message)
    print("Step 3) Clarification:", clarifying)

    final_answer = compose_final_answer(snippet, clarifying)
    print("Step 4) Final Answer:\n", final_answer)

    qc = quality_check(final_answer)
    print("Step 5) Quality Check:", qc)

    return {
        "intent": intent_info,
        "snippet": snippet,
        "clarifying": clarifying,
        "final_answer": final_answer,
        "quality": qc
    }

# --- Try it with some JCPenney examples ---
# Example 1: Credit card issue
user_message = "I need help with my JCPenney credit card payment."
result = run_support_chain(user_message)

print("\n\n--- Another Example ---\n")
# Example 2: Returns
user_message2 = "How many days do I have to return something to JCPenney?"
result2 = run_support_chain(user_message2)

USER MESSAGE: I need help with my JCPenney credit card payment.
------------------------------------------------------------
Step 1) Intent: {'intent': 'credit_card', 'confidence': 0.92}
Step 2) FAQ Snippet: For JCPenney credit card (regular or Mastercard) payments or billing issues, log in to your JCPenney Credit Card Account online. You can update your payment method under Billing → Update Payment Method.
Step 3) Clarification: Could you tell me if this is about your JCPenney Credit Card, a return, or an order tracking request?
Step 4) Final Answer:
 Thanks for contacting JCPenney Customer Support! Here's what to do:
- For JCPenney credit card (regular or Mastercard) payments or billing issues, log in to your JCPenney Credit Card Account online. You can update your payment method under Billing → Update Payment Method.
If you run into trouble, reply here and we’ll be happy to assist further.

Before we proceed: Could you tell me if this is about your JCPenney Credit Card, a return, or